<a href="https://colab.research.google.com/github/nyb05/iitgcapstone/blob/main/iitgcapstonenb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

df = pd.read_csv("https://raw.githubusercontent.com/nyb05/iitgcapstone/refs/heads/main/dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp')

In [ ]:
lot_df = df[df['SystemCodeNumber'] == 'BHMBCCMKT01'].copy()
lot_df['OccupancyRatio'] = lot_df['Occupancy'] / lot_df['Capacity']

base_price = 10.0
alpha = 2.0

prices = [base_price]
for ratio in lot_df['OccupancyRatio'].iloc[1:]:
    prices.append(prices[-1] + alpha * ratio)

lot_df = lot_df.iloc[1:].copy()
lot_df['Price_Model1'] = prices[1:]


In [ ]:
lot_df['TrafficConditionNearby'] = lot_df['TrafficConditionNearby'].replace('average', 'medium')

vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.5}
traffic_weights = {'low': 0.5, 'medium': 1.0, 'high': 1.5}
lot_df['VehicleTypeWeight'] = lot_df['VehicleType'].map(vehicle_weights)
lot_df['TrafficLevel'] = lot_df['TrafficConditionNearby'].map(traffic_weights)

α, β, γ, δ, ε = 2.0, 0.5, 1.0, 1.5, 1.0
lot_df['Demand'] = (
    α * lot_df['OccupancyRatio'] +
    β * lot_df['QueueLength'] -
    γ * lot_df['TrafficLevel'] +
    δ * lot_df['IsSpecialDay'] +
    ε * lot_df['VehicleTypeWeight']
)

min_d, max_d = lot_df['Demand'].min(), lot_df['Demand'].max()
lot_df['NormalizedDemand'] = (lot_df['Demand'] - min_d) / (max_d - min_d)

λ = 0.75
lot_df['Price_Model2'] = base_price * (1 + λ * lot_df['NormalizedDemand'])
lot_df['Price_Model2'] = lot_df['Price_Model2'].clip(0.5 * base_price, 2.0 * base_price)

In [ ]:
timepoint = df['Timestamp'].iloc[100]
active_df = df[df['Timestamp'] == timepoint].copy()
active_df['OccupancyRatio'] = active_df['Occupancy'] / active_df['Capacity']
active_df['Price_Model3'] = base_price + alpha * active_df['OccupancyRatio']

target_lot = active_df.iloc[0]
target_coords = (target_lot['Latitude'], target_lot['Longitude'])
target_price = target_lot['Price_Model3']

def competitive_adjustment(row):
    dist = geodesic(target_coords, (row['Latitude'], row['Longitude'])).meters
    price_diff = row['Price_Model3'] - target_price
    if dist < 300 and price_diff < 0:
        return target_price - 0.5
    elif dist < 300 and price_diff > 1:
        return min(target_price + 0.5, 2 * base_price)
    else:
        return target_price

active_df['Price_Model3_Updated'] = active_df.apply(competitive_adjustment, axis=1)
